In [59]:
from hmmlearn import hmm
import numpy as np

In [60]:
seed = 93
np.random.seed(seed)

In [61]:
states = ["left_lane", "right_lane"]
observations = ["low_speed", "high_speed"]

In [62]:
discrete_model = hmm.MultinomialHMM(n_components=2,
                                    algorithm='viterbi',  # Decoder algorithm.
                                    # algorithm='map'  # todo: what does MaP?
                                    random_state=seed,
                                    n_iter=10,
                                    tol=0.01  # EM Convergence threshold (gain in log-likelihood)
                                   )

# Q1 - How to easily estimate the parameters of our HMM?
Here are the results of the estimation procedure based on **counting from the labelled data**. It constitutes the **MLE**.

In [63]:
discrete_model.startprob_ = np.array(
    [1/3, 2/3]
)
discrete_model.transmat_ = np.array([
    [0.6, 0.4],  # P(state_t+1|state_t=state_0)
    [0.2, 0.8]]  # P(state_t+1|state_t=state_1)
)
discrete_model.emissionprob_ = np.array(
    [[0.4, 0.6],  # P(obs|state_0)
     [0.8, 0.2]]  # P(obs|state_1)
)

### Sampling
Generate samples from the HMM 

In [64]:
# use the parameters to generates samples
# using a large sample size to estimate the stationary state distributions: 1/3 vs 2/3
# (n_samples, n_features)
Obss, States = discrete_model.sample(100000)
States = States.flatten()
Obss = Obss.flatten()
from collections import Counter
print(Counter(States))
print(Counter(Obss))

Counter({1: 66134, 0: 33866})
Counter({0: 66520, 1: 33480})


### Stationary distribution from the transition matrix
The **stationary distribution** is a **left eigenvector** (as opposed to the usual right eigenvectors) of the **transition matrix**.

In [65]:
eigvals, eigvecs = np.linalg.eig(discrete_model.transmat_.T)
eigvec = np.real_if_close(eigvecs[:, np.argmax(eigvals)])
print(eigvecs)
print(eigvals)
print(eigvec)
# normalisation
stat_distr = eigvec / eigvec.sum()
print(stat_distr)

# The stationary distribution is proportional to the left-eigenvector
# associated with the largest eigenvalue (i.e., 1) of the transition matrix
x1 = np.asarray([-0.70710678, 0.70710678 ])
res1 = np.dot(discrete_model.transmat_.T, x1)
print(res1)
print(1/0.28284271)

x2 = np.asarray([-0.4472136, -0.89442719])  # invariant
res2 = np.dot(discrete_model.transmat_.T, x2)
print(res2)

[[-0.70710678 -0.4472136 ]
 [ 0.70710678 -0.89442719]]
[0.4 1. ]
[-0.4472136  -0.89442719]
[0.33333333 0.66666667]
[-0.28284271  0.28284271]
3.5355339368654755
[-0.4472136  -0.89442719]


In [66]:
0.4472136 /0.70710678

0.6324555394589767

# Q2 - Given a single speed observation, what is the probability for the car to be in each of the two lanes?

# Q3 - What is the probability to observe a particular sequence of speed measurements?

In [67]:
# log-likelihood

# Q4 - Given a sequence of speed observations, what is the most likely current lane?

# Q5 - Given a sequence of speed observations, what is the most likely underlying lane sequence?

### Decoding
Find most likely state sequence corresponding to the observation sequence

In [68]:
obs_sequence = np.array([[0, 1, 0]]).T
logprob, state_sequence = discrete_model.decode(obs_sequence)

# Log probability of the produced state sequence
for o, s in zip(obs_sequence.T[0], state_sequence):
    print("{} -> {}".format(states[int(s)], observations[int(o)]))
print("\nprob = {}\nlog_prob = {}".format(np.exp(logprob), logprob))

right_lane -> low_speed
right_lane -> high_speed
right_lane -> low_speed

prob = 0.05461333333333335
log_prob = -2.9074772257991035


# Q6 - How to estimate the parameters of our HMM when no state annotations are present in the training data?

### Learning
Estimate HMM model parameters based on observation data

In [69]:
obs_sequence = np.array([[0, 1, 0]]).T
new_model = discrete_model.fit(obs_sequence)
print(new_model.transmat_)

[[3.70410311e-39 1.00000000e+00]
 [1.00000000e+00 2.77040565e-10]]


In [70]:
# sample with sample()
# train an HMM by calling the fit() method.
# the inferred optimal hidden states can be obtained by calling predict() method.
#     Viterbi algorithm ("viterbi"), and maximum a posteriori estimation ("map") are supported
# the score of the model can be calculated by the score() method.

In [ ]:
eigenvalue of the matrix